In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
import numpy as np
import string
import random
from SoundsLike.SoundsLike import Search
import Levenshtein

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shahzainmehboob/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/shahzainmehboob/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:
df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
#df = pd.read_csv("data/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [4]:
df.head()

,-DOCSTART-,-X-,-X-.1,O
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O


In [5]:
df = df[df['O'].notna()]

In [6]:
df.drop(['-X-', '-X-.1'], axis=1, inplace=True)

In [7]:
df.columns = ['Word', 'Tag']

In [8]:
df.head()

,Word,Tag
0,EU,B-ORG
1,rejects,O
2,German,B-MISC
3,call,O
4,to,O


In [9]:
#df.drop(df[df['Word'] == "-DOCSTART-"].index, axis=0, inplace=True)

In [10]:
df['Tag'].unique()

array(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG',
       'I-MISC', 'I-LOC'], dtype=object)

In [11]:
tags = df['Tag'].values
words = df['Word'].values

In [12]:
new_tags = []
for t in tags:
    if t == "B-ORG":
        new_tags.append("ORG")
    elif t == "B-PER":
        new_tags.append("PER")
    elif t == "I-PER":
        new_tags.append("PER")
    elif t == "B-LOC":
        new_tags.append("LOC")
    elif t == "I-ORG":
        new_tags.append("ORG")
    elif t == "I-LOC":
        new_tags.append("LOC")
    else:
        new_tags.append("O")

In [13]:
df.drop(['Tag'], axis=1, inplace=True)
df.drop(['Word'], axis=1, inplace=True)

In [14]:
df['Tag'] = new_tags
df['Word'] = words

In [15]:
len(df)

202388

In [16]:
sentence_no = 0
sentence = []
for w in words:
    sentence.append(sentence_no)
    if w == ".":
        sentence_no = sentence_no + 1

In [17]:
df.insert(0, 'Sentence #', sentence)

In [18]:
df.head(270)

,Sentence #,Tag,Word
0,0,ORG,EU
1,0,O,rejects
2,0,O,German
3,0,O,call
4,0,O,to
5,0,O,boycott
6,0,O,British
7,0,O,lamb
8,0,O,.
9,1,PER,Peter


In [19]:
df['Tag'].unique()

array(['ORG', 'O', 'PER', 'LOC'], dtype=object)

In [20]:
len(df)

202388

In [21]:
dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if tag != "O":
        error = []
        try:
            x = Search.perfectHomophones(word.lower())
            for d in x:
                err = Levenshtein.distance(word.lower(), d)
                error.append(err)
                word = x[error.index(max(error))]
        except ValueError:
            pass 
        dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))
df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

KeyboardInterrupt: 

In [ ]:
df.head(100)

In [ ]:
g_test = df.groupby("Sentence #")
x = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(map(str,sdf.Tag)))})

In [ ]:
x.head()

In [ ]:
count = []
for sentence in x['Sentence'].values.tolist():
    count.append(len(sentence.split(" ")))

In [ ]:
count.sort(reverse=True)

In [ ]:
count

In [ ]:
from statistics import mean
avg = mean(count)
avg

In [ ]:
import re

def decontracted(phrase):
    # general
    phrase = re.sub(r" \'s", "\'s", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" n't", "n't", phrase)
    phrase = re.sub(r" \'re", "\'re", phrase)
    phrase = re.sub(r" \'d", "\'d", phrase)
    phrase = re.sub(r" \'ll", "\'ll", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" \'ve", "\'ve", phrase)
    phrase = re.sub(r" \'m", "\'m", phrase)
    return phrase

In [ ]:
p_data = []
sentence_no = 0
new_df = []
for sentence, tags in zip(x['Sentence'].values.tolist(), x['Tag'].values.tolist()):
    words = sentence.split(" ")
    ind = [ i for i, word in enumerate(words) if word == "'s" or word == "'t" or word == "'re" or word == "'d" or word == "'ll" or word == "'t" or word == "'ve" or word == "'m" or word == "n't"]
    labels = tags.split(",")
    sentence = " ".join(words)
    sentence = decontracted(sentence)
    tags = [ elem for i, elem in enumerate(labels) if i not in ind]
    stripped = sentence.translate(str.maketrans('', '', string.punctuation))
    stripped = stripped.split(" ")
    for word, label in zip(stripped, tags):
        p_data.append((sentence_no, word, label))
    sentence_no = sentence_no + 1

In [ ]:
df = pd.DataFrame(p_data, columns=['Sentence #', 'Word', 'Tag'])

In [ ]:
#df.to_csv("../../data/conll03/new_conll_train_preprocessed_with_punctuation_with_broken_entities2.csv")
df.to_csv("../../data/conll03/new_conll_train_preprocessed_without_punctuation_with_broken_entities2.csv")

In [ ]:
x = Search.closeHomophones("brussels")

In [ ]:
x